In [1]:
import pandas as pd
from decimal import Decimal
import json
import numpy as np
from pprint import pprint
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.2-BETA2 (03/Jan/2022)


# Carbon Simulation - Demo-5-7 Simulator Import_Export NEW

Here we demonstrate the preferred setup to import and export json test files from the simulator using the new format

In [2]:
# Import Code
def load_json(filename):     
    '''
    :filename:  the convention is to specify the trade by source or trade by target in the filename i.e. tradeByTargetAmount.json
                this is necessary as the `is_by_target` field specifies the trade direction for the actionList
    '''   
    f = open(filename , "r")
    bm = f.read()
    loaded_json = json.loads(bm)
    f.close()
    return(loaded_json)

def decimalize_dict(dicta):
    deci_dict = {}
    expected_dict = {}
    for k,v in dicta.items():
        if k == 'expected':
            for i,j in dicta['expected'].items():
                expected_dict[i] = Decimal(j)
        else:    
            if k != 'token':
                v = Decimal(v)
            deci_dict[k] = v
    deci_dict['expected'] = expected_dict
    return(deci_dict)

def calc_y_int(liquidity, highestRate, lowestRate, marginalRate):
    return(liquidity * (highestRate.sqrt() - lowestRate.sqrt()) / (marginalRate.sqrt() - lowestRate.sqrt()))

def parse_order(order0, order_count):
    order0 = decimalize_dict(order0)
    order0['order'] = order_count
    order0_expected = {k:v for k,v in order0['expected'].items()}
    del order0['expected']
    order0_df = pd.DataFrame.from_dict(order0, 'index', columns=[f'{order_count}'])
    order0_expected_df = pd.DataFrame.from_dict(order0_expected, 'index', columns=[f'{order_count}'])
    order0_expected_df.index = ["expected_"+x for x in order0_expected_df.index]
    order0_df = pd.concat([order0_df, order0_expected_df], axis=0)
    return(order0_df)

def parse_strategy(strategy, strat_count, order_count):
    orders = pd.DataFrame()
    orderList = strategy['orders']
    for order in orderList:
        order_df = parse_order(order, order_count)
        orders = pd.concat([orders,order_df.T])
        order_count += 1
    orders['strategy'] = strat_count
    strat_count += 1
    return(orders, strat_count, order_count)

def format_json_as_df(input_json, scenario_index = 0):
    '''
    :json:             the loaded json file
    :scenario_index:   each json file has multiple scenarios, make a selection
    '''
    actionList = input_json[scenario_index]['tradeActions']
    is_by_target = input_json[scenario_index]['tradeByTargetAmount']
    sourceToken = input_json[scenario_index]['sourceToken']
    targetToken = input_json[scenario_index]['targetToken']
    
    print(f'The number of scenarios is {len(input_json)}')
    print(f'You selected scenario: {scenario_index}')  
    print(f'The number of actions in this scenario are: {len(actionList)}')

    if scenario_index >= len(input_json):
        print('Please selected a valid scenario_index')
        return()
    else:
        strats = pd.DataFrame()
        strat_count = 0
        order_count = 0
        for strategy in input_json[scenario_index]['strategies']:
            orders, strat_count, order_count = parse_strategy(strategy, strat_count, order_count)
            strats = pd.concat([strats, orders])
        strats = strats[["token","order","strategy","liquidity","lowestRate","highestRate","marginalRate","expected_liquidity","expected_lowestRate","expected_highestRate","expected_marginalRate"]]
        return(strats, actionList, is_by_target, sourceToken, targetToken)

def setup_initial_orders(strats, sourceToken, targetToken):
    FastSim = CarbonSimulatorUI(pair=f"{sourceToken}/{targetToken}", verbose=False, matching_method='exact', raiseonerror=True)

    if strats.token[0] == sourceToken:
        for i in strats.order[::2]:
            FastSim.add_strategy(
                tkn=f'{sourceToken}',
                amt_sell=Decimal(str(strats.liquidity[i])),
                psell_start=1/strats.highestRate[i],
                psell_end=1/strats.lowestRate[i],
                psell_marginal=1/strats.marginalRate[i],
                amt_buy=strats.liquidity[i+1],
                pbuy_start=strats.highestRate[i+1],
                pbuy_end=strats.lowestRate[i+1],
                pbuy_marginal=strats.marginalRate[i+1],
            )
    else:
        for i in strats.order[::2]:
            FastSim.add_strategy(
                tkn=f'{targetToken}',
                amt_sell=Decimal(str(strats.liquidity[i])),
                psell_start=strats.highestRate[i],
                psell_end=strats.lowestRate[i],
                psell_marginal=strats.marginalRate[i],
                amt_buy=strats.liquidity[i+1],
                pbuy_start=1/strats.highestRate[i+1],
                pbuy_end=1/strats.lowestRate[i+1],
                pbuy_marginal=1/strats.marginalRate[i+1],
            )

    initial_orders_import = FastSim.state()['orders']
    return(FastSim, initial_orders_import)

def parse_actions(Sim, actionList, strats, is_by_target, sourceToken, targetToken):
    '''
    parse_actions processes trades from the actionList

    :Sim:           the specific simulator environment to trade against
    :actionList:    the list of actions provides from the test json
    :strats:        the preprocessed dataframe of strategies from the test json
    :is_by_target:  the specification of trading by target or by source
    '''
    print(f"Trading by Target: {is_by_target}; Source Token is {sourceToken}; Target Token is {targetToken}")

    for action in actionList:
        strategyId = int(action['strategyId']) - 1
        targetStrategy = strats.query(f"strategy=={strategyId}")
        tokenAmount = Decimal(action['amount'])
        # print(strategyId, tokenAmount)
        if not is_by_target:
            targetOrderId = targetStrategy.query(f"token=='{targetToken}'")['order'].values[0]
            Sim.amm_buys(sourceToken, tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_source
            # print(sourceToken, targetOrderId, tokenAmount)
        else:
            targetOrderId = targetStrategy.query(f"token=='{targetToken}'")['order'].values[0]
            Sim.amm_sells(targetToken, tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_target
            # print(targetToken, targetOrderId, tokenAmount)

def single_assertion(input_json, final_orders_import, scenario_index, strat_index, accuracy, order, input_variable, final_variable, print=False):
    '''
    Asserts that the input_variable and the final_variable are exact given a defined accuracy
    '''
    if print:
        print(f"{(final_orders_import.loc[strat_index,final_variable]):.{accuracy}f}" , f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][order]['expected'][input_variable]):.{accuracy}f}")
    assert(f"{(final_orders_import.loc[strat_index,final_variable]):.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][order]['expected'][input_variable]):.{accuracy}f}")

def single_assertion_flipped(input_json, final_orders_import, scenario_index, strat_index, accuracy, order, input_variable, final_variable, print=False):
    '''
    Asserts that the input_variable and the inverse of the final_variable are exact given a defined accuracy
    '''
    if print:
        print(f"{1/(final_orders_import.loc[strat_index,final_variable]):.{accuracy}f}" , f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][order]['expected'][input_variable]):.{accuracy}f}")
    assert(f"{1/(final_orders_import.loc[strat_index,final_variable]):.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][order]['expected'][input_variable]):.{accuracy}f}")

def multiLevel_assertion(input_json, final_orders_import, scenario_index, strat_index, accuracy, input_variable, final_variable):
    '''
    Multi-level assertion - Asserts that the input_variable and the final_variable (or its inverse) are exact given a defined accuracy

    Specific use-case: When strategies are entered into the simulator `p_marg` may be inverted depending on how the pair is quoted.
    Such, exactly one combination should be able to be asserted.
    '''
    try:
        single_assertion(input_json, final_orders_import, scenario_index, strat_index, accuracy, 0, input_variable, final_variable, print=False)
    except:
        single_assertion_flipped(input_json, final_orders_import, scenario_index, strat_index, accuracy, 0, input_variable, final_variable, print=False)

    # equivalent for the other order in the strategy (strat_index increment by one and the order set to one)
    try:
        single_assertion(input_json, final_orders_import, scenario_index, strat_index+1, accuracy, 1, input_variable, final_variable, print=False)
    except:
        single_assertion_flipped(input_json, final_orders_import, scenario_index, strat_index+1, accuracy, 1, input_variable, final_variable, print=False)


def multiLevel_assertion_two_var(input_json, final_orders_import, scenario_index, strat_index, accuracy, input_variable, final_variable_1, final_variable_2):
    '''
    Two-variable multi-level assertion - Asserts that the input_variable and at least one of the final_variable_1 (or its inverse) or final_variable_2 (or its inverse)
    are exact given a defined accuracy

    Specific use-case: When strategies are entered into the simulator, `p_start` and `p_end` are derived from `p_a` and `p_b` however these values may be swapped or
    inverted depending on the way the pair is quoted and the order they are provided. Nevertheless, exactly one combination should be able to be asserted.
    '''
    try:
        single_assertion(input_json, final_orders_import, scenario_index, strat_index, accuracy, 0, input_variable, final_variable_1, print=False)
    except:
        try:
            single_assertion_flipped(input_json, final_orders_import, scenario_index, strat_index, accuracy, 0, input_variable, final_variable_1, print=False)
        except:
            try:
                single_assertion(input_json, final_orders_import, scenario_index, strat_index, accuracy, 0, input_variable, final_variable_2, print=False)
            except:
                single_assertion_flipped(input_json, final_orders_import, scenario_index, strat_index, accuracy, 0, input_variable, final_variable_2, print=False)
    
    # equivalent for the other order in the strategy (strat_index increment by one and the order set to one)
    try:
        single_assertion(input_json, final_orders_import, scenario_index, strat_index+1, accuracy, 1, input_variable, final_variable_1, print=False)
    except:
        try:
            single_assertion_flipped(input_json, final_orders_import, scenario_index, strat_index+1, accuracy, 1, input_variable, final_variable_1, print=False)
        except:
            try:
                single_assertion(input_json, final_orders_import, scenario_index, strat_index+1, accuracy, 1, input_variable, final_variable_2, print=False)
            except:
                single_assertion_flipped(input_json, final_orders_import, scenario_index, strat_index+1, accuracy, 1, input_variable, final_variable_2, print=False)

def verify_json(filename, accuracy=7):
    input_json = load_json(filename)
    for scenario_index in range(len(input_json)):
        print("\n")
        strats, actionList, is_by_target, sourceToken, targetToken = format_json_as_df(input_json, scenario_index = scenario_index)
        FastSim, initial_orders_import = setup_initial_orders(strats, sourceToken, targetToken)
        parse_actions(FastSim, actionList, strats, is_by_target, sourceToken, targetToken)
        final_orders_import = FastSim.state()['orders']
        for strat_index in final_orders_import.index[::2]:
            # print(strat_index)
            single_assertion(input_json, final_orders_import, scenario_index, strat_index, accuracy, order = 0, input_variable ='liquidity', final_variable='y')
            single_assertion(input_json, final_orders_import, scenario_index, strat_index+1, accuracy, order = 1, input_variable ='liquidity', final_variable='y')
            multiLevel_assertion(input_json, final_orders_import, scenario_index, strat_index, accuracy, 'marginalRate', 'p_marg')
            multiLevel_assertion_two_var(input_json, final_orders_import, scenario_index, strat_index, accuracy, 'lowestRate', 'p_start', 'p_end')
            multiLevel_assertion_two_var(input_json, final_orders_import, scenario_index, strat_index, accuracy, 'highestRate', 'p_start', 'p_end')

    print("\n----------All Tests Passed!----------\n")

def verify_json2(filename, accuracy=12):
    misses = {}
    misses['liquidity'] = {}
    misses['marginalPrice'] = {}
    input_json = load_json(filename)
    for scenario_index in range(len(input_json)):
        print("\n")
        strats, actionList, is_by_target, sourceToken, targetToken = format_json_as_df(input_json, scenario_index = scenario_index)
        FastSim, initial_orders_import = setup_initial_orders(strats, sourceToken, targetToken)
        parse_actions(FastSim, actionList, strats, is_by_target, sourceToken, targetToken)
        final_orders_import = FastSim.state()['orders']
        sub_misses = []
        sub_misses_p = []
        for strat_index in final_orders_import.index[::2]:
            if not f"{final_orders_import.y[strat_index]:.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][0]['expected']['liquidity']):.{accuracy}f}":
                sub_misses += [(f"{final_orders_import.y[strat_index]:.{accuracy}f}", f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][0]['expected']['liquidity']):.{accuracy}f}")]
            
            if not f"{final_orders_import.y[strat_index+1]:.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][1]['expected']['liquidity']):.{accuracy}f}":
                sub_misses += [(f"{final_orders_import.y[strat_index+1]:.{accuracy}f}", f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][1]['expected']['liquidity']):.{accuracy}f}")]

            if f"{(final_orders_import.p_marg[strat_index]):.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][0]['expected']['marginalRate']):.{accuracy}f}":
                pass
            elif f"{1/(final_orders_import.p_marg[strat_index]):.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][0]['expected']['marginalRate']):.{accuracy}f}":
                 pass
            else:
                sub_misses_p += [(f"{1/(final_orders_import.p_marg[strat_index]):.{accuracy}f}", f"{(final_orders_import.p_marg[strat_index]):.{accuracy}f}",  f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][0]['expected']['marginalRate']):.{accuracy}f}")]

            if f"{(final_orders_import.p_marg[strat_index+1]):.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][1]['expected']['marginalRate']):.{accuracy}f}":
                pass
            elif f"{1/(final_orders_import.p_marg[strat_index+1]):.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][1]['expected']['marginalRate']):.{accuracy}f}":
                pass
            else:
                sub_misses_p += [(f"{1/(final_orders_import.p_marg[strat_index+1]):.{accuracy}f}", f"{(final_orders_import.p_marg[strat_index+1]):.{accuracy}f}", f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][1]['expected']['marginalRate']):.{accuracy}f}")]
        misses['liquidity'][scenario_index] = sub_misses
        misses['marginalPrice'][scenario_index] = sub_misses_p
    return(misses)

# Export Code
def generate_json_actions_new(Sim, trade_a):

    json_actions = {}
    sub_actions_list = []

    if trade_a['success'] & trade_a['trades'].exec.all():
        subdf = trade_a['trades'].query('subid!="A"').copy()
        subdf.reset_index(inplace=True, drop=True)
        trade_is_by_target = trade_a['is_by_target']
        tkn = trade_a['tkn_sold']
        sourceToken = Sim.carbon_pair.other(tkn)
        targetToken = tkn

        print(f'trade_is_by_target: {trade_is_by_target}')
        for i in subdf.index:
            sub_actions = {}
            sub_actions['strategyId'] = f"{int((subdf.routeix[i]//2) + 1)}" # plus one to encode for contracts
            if trade_is_by_target:
                sub_actions['amount'] = f"{subdf.amt1[i]:.12f}"
            else:
                sub_actions['amount'] = f"{subdf.amt2[i]:.12f}"
            sub_actions_list += [sub_actions]
            
    json_actions['tradeActions'] = sub_actions_list
    return(json_actions, trade_is_by_target, sourceToken, targetToken)

def generate_json_strategies_new(initial_orders, final_orders):
    json_strats = {}
    json_orders = {}
    json_orders_list = []
    sub_dict_list = []
    for i in initial_orders.index[::2]:
        json_orders = {} 
        sub_dict0 = {}
        sub_dict0['token'] = initial_orders.tkn[i]
        sub_dict0['liquidity'] = f"{initial_orders.y[i]:.12f}"
        sub_dict0['lowestRate'] = f"{1/(initial_orders.p_end[i]):.12f}"  # - 0.000000001
        sub_dict0['highestRate'] = f"{(1/initial_orders.p_start[i]):.12f}" # + 0.000000001
        sub_dict0['marginalRate'] = f"{(1/initial_orders.p_marg[i]):.12f}"

        sub_dict0['expected'] = {}
        sub_dict0['expected']['liquidity'] = f"{final_orders.y[i]:.12f}"
        sub_dict0['expected']['lowestRate'] = f"{1/(final_orders.p_end[i]):.12f}"  # - 0.000000001
        sub_dict0['expected']['highestRate'] = f"{(1/final_orders.p_start[i]):.12f}" # + 0.000000001
        sub_dict0['expected']['marginalRate'] = f"{(1/final_orders.p_marg[i]):.12f}"
        
        sub_dict1 = {}   
        
        sub_dict1['token'] = initial_orders.tkn[i+1]
        sub_dict1['liquidity'] = f"{initial_orders.y[i+1]:.12f}" 
        sub_dict1['lowestRate'] = f"{((initial_orders.p_end[i+1])):.12f}" # + 0.000000001
        sub_dict1['highestRate'] = f"{((initial_orders.p_start[i+1])):.12f}" # - 0.000000001
        sub_dict1['marginalRate'] = f"{(initial_orders.p_marg[i+1]):.12f}"
        
        sub_dict1['expected'] = {}
        sub_dict1['expected']['liquidity'] = f"{final_orders.y[i+1]:.12f}" 
        sub_dict1['expected']['lowestRate'] = f"{((final_orders.p_end[i+1])):.12f}" # + 0.000000001
        sub_dict1['expected']['highestRate'] = f"{((final_orders.p_start[i+1])):.12f}" # - 0.000000001
        sub_dict1['expected']['marginalRate'] = f"{(final_orders.p_marg[i+1]):.12f}"

        json_orders['orders'] = [sub_dict0,sub_dict1]
        json_orders_list += [json_orders]
    json_strats['strategies'] = json_orders_list
    return(json_strats)

def format_simulator_json_new(json_strats, json_actions, trade_is_by_target, sourceToken, targetToken):
    json_simulator = []
    simulator_setup = {}
    simulator_setup['tradeByTargetAmount'] = True if trade_is_by_target else False
    simulator_setup['sourceToken'] = sourceToken
    simulator_setup['targetToken'] = targetToken
    simulator_setup.update({**json_strats, **json_actions})
    json_simulator += [simulator_setup]
    return(json_simulator)



# To Import Json and Verify against simulator
- Asserts that expected liquidity and marginalRate match the expected results to a given accuracy threshold

In [3]:
filename = '../benchmark/ArbitraryTrade.spec.json'
verify_json(filename, accuracy=7)



The number of scenarios is 200
You selected scenario: 0
The number of actions in this scenario are: 1
Trading by Target: False; Source Token is TKN0; Target Token is TKN1


The number of scenarios is 200
You selected scenario: 1
The number of actions in this scenario are: 2
Trading by Target: False; Source Token is TKN0; Target Token is TKN1


The number of scenarios is 200
You selected scenario: 2
The number of actions in this scenario are: 3
Trading by Target: False; Source Token is TKN0; Target Token is TKN1


The number of scenarios is 200
You selected scenario: 3
The number of actions in this scenario are: 4
Trading by Target: False; Source Token is TKN0; Target Token is TKN1


The number of scenarios is 200
You selected scenario: 4
The number of actions in this scenario are: 5
Trading by Target: False; Source Token is TKN0; Target Token is TKN1


The number of scenarios is 200
You selected scenario: 5
The number of actions in this scenario are: 6
Trading by Target: False; Sourc

- Tests every expected liquidity and marginalRate match the expected results against a given accuracy threshold and returns a dict of the failures

In [4]:
%%capture
misses = verify_json2(filename, accuracy=12)

In [5]:
print('Liquidity')
for i in misses['liquidity'].keys():
    if len(misses['liquidity'][i])>0:
        print(i, misses['liquidity'][i])
print('\n')
print('Marginal Prices')
for i in misses['marginalPrice'].keys():
    if len(misses['marginalPrice'][i])>0:
        print(i, misses['marginalPrice'][i])

Liquidity
17 [('1260992.583529681433', '1260992.583529681200')]
58 [('2971148.877221512608', '2971148.877221512143')]
59 [('3601511.587727685459', '3601511.587727684993')]
126 [('439102.575111364480', '439102.575111364422')]
169 [('387527.534839537111', '387527.534839537053')]
174 [('1213843.436529699480', '1213843.436529699247')]


Marginal Prices
59 [('0.029701286528', '33.668575233133', '33.668575233132')]


# To Create New Scenario and Export to Json

## Initialize a simulator and add some strategies

In [6]:
# New Sim
FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='exact', raiseonerror=True)

In [7]:
# Add strategies
FastSim.add_strategy("ETH", 10, 2000, 2500, 10000, 1040, 1020, "ETH/USDC", 2100, 1030)
FastSim.add_strategy("ETH", 10, 2010, 2810, 10000, 1510, 1210, "ETH/USDC", 2100, 1250)
FastSim.add_strategy("ETH", 10, 2020, 2220, 10000, 1060, 1050, "ETH/USDC", 2200, 1055)
FastSim.add_strategy("ETH", 10, 2030, 2060, 10000, 1750, 1055, "ETH/USDC", 2035, 1700)
FastSim.add_strategy("ETH", 10, 2040, 2080, 10000, 1250, 1040, "ETH/USDC", 2050, 1050)

# Snapshot the simulator orders
initial_orders = FastSim.state()['orders']

In [8]:
initial_orders

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,12.958031,10.0,ETH,False,2000.0,2500.0,2100.0,USDC per ETH,1
1,1,ETHUSDC,USDC,19951.572996,10000.0,USDC,False,1040.0,1020.0,1030.0,USDC per ETH,0
2,2,ETHUSDC,ETH,11.633958,10.0,ETH,False,2010.0,2810.0,2100.0,USDC per ETH,3
3,3,ETHUSDC,USDC,71432.102199,10000.0,USDC,False,1510.0,1210.0,1250.0,USDC per ETH,2
4,4,ETHUSDC,ETH,106.581944,10.0,ETH,False,2020.0,2220.0,2200.0,USDC per ETH,5
5,5,ETHUSDC,USDC,19976.331261,10000.0,USDC,False,1060.0,1050.0,1055.0,USDC per ETH,4
6,6,ETHUSDC,ETH,12.022136,10.0,ETH,False,2030.0,2060.0,2035.0,USDC per ETH,7
7,7,ETHUSDC,USDC,10687.914210,10000.0,USDC,False,1750.0,1055.0,1700.0,USDC per ETH,6
8,8,ETHUSDC,ETH,13.382254,10.0,ETH,False,2040.0,2080.0,2050.0,USDC per ETH,9
9,9,ETHUSDC,USDC,200831.310629,10000.0,USDC,False,1250.0,1040.0,1050.0,USDC per ETH,8


## Do a trade and process it into an actionList

In [9]:
# Do a trade
trade_a = FastSim.amm_sells('ETH', 29, execute=True) # route_trade_by_target
# trade_a = FastSim.amm_buys('ETH', 15, execute=True) # route_trade_by_source
# trade_a = FastSim.amm_sells('USDC', 50000, execute=True) # route_trade_by_target
# trade_a = FastSim.amm_buys('USDC', 30000, execute=True) # route_trade_by_source

# Generate the actions component of the json
json_actions, trade_is_by_target, sourceToken, targetToken  = generate_json_actions_new(FastSim, trade_a)
sourceToken, targetToken

trade_is_by_target: True


('USDC', 'ETH')

In [10]:
# Snapshot the orders
final_orders = FastSim.state()['orders']
final_orders

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,12.958031,7.024559,ETH,False,2000.0,2500.0,2208.350060,USDC per ETH,1
1,1,ETHUSDC,USDC,19951.572996,16407.593124,USDC,False,1040.0,1020.0,1036.433236,USDC per ETH,0
2,2,ETHUSDC,ETH,11.633958,8.166987,ETH,False,2010.0,2810.0,2208.350060,USDC per ETH,3
3,3,ETHUSDC,USDC,71432.102199,13947.381198,USDC,False,1510.0,1210.0,1265.968579,USDC per ETH,2
4,4,ETHUSDC,ETH,106.581944,5.808453,ETH,False,2020.0,2220.0,2208.350060,USDC per ETH,5
5,5,ETHUSDC,USDC,19976.331261,19238.885565,USDC,False,1060.0,1050.0,1059.629998,USDC per ETH,4
6,6,ETHUSDC,ETH,12.022136,0.000000,ETH,False,2030.0,2060.0,2060.000000,USDC per ETH,7
7,7,ETHUSDC,USDC,30474.618434,30474.618434,USDC,False,1750.0,1055.0,1750.000000,USDC per ETH,6
8,8,ETHUSDC,ETH,13.382254,0.000000,ETH,False,2040.0,2080.0,2080.000000,USDC per ETH,9
9,9,ETHUSDC,USDC,200831.310629,30649.455199,USDC,False,1250.0,1040.0,1070.800866,USDC per ETH,8


## Generate the strategies component of the json

In [11]:
# Generate the strategies component of the json
json_strats = generate_json_strategies_new(initial_orders, final_orders)

## Combine and format the jsons

In [12]:
json_simulator = format_simulator_json_new(json_strats, json_actions, trade_is_by_target, sourceToken, targetToken)
# pprint(json_simulator, sort_dicts=False)

## Optionally Save

In [18]:
from datetime import datetime
str_date = datetime.today().strftime("%Y%m%d")
unique_id = datetime.now().strftime("%f")
    
export_filename = f"../other/{str_date}_test_{unique_id}.json"

out_file = open(export_filename, "w")
json.dump(json_simulator, out_file, indent=4)
out_file.close()

print(f'Saved as {export_filename}')

Saved as ../other/20230109_test_462256.json


# To Import json and Evaluate Step-wise

First we load the json test file and return a df of the strategies and a list of actions

** If you want to use a specific scenario you need to specify the scenario_index here **

In [19]:
# Specify the scenario_index for a different scenario
filename= export_filename
# filename = 'ArbitraryTrade.json'
scenario_index = 0

input_json = load_json(filename)
strats, actionList, is_by_target, sourceToken, targetToken = format_json_as_df(input_json, scenario_index = scenario_index)
strats

The number of scenarios is 1
You selected scenario: 0
The number of actions in this scenario are: 5


,token,order,strategy,liquidity,lowestRate,highestRate,marginalRate,expected_liquidity,expected_lowestRate,expected_highestRate,expected_marginalRate
0,ETH,0,0,10.000000000000,0.000400000000,0.000500000000,0.000476190476,7.024559153479,0.000400000000,0.000500000000,0.000452826759
1,USDC,1,0,10000.000000000000,1020.000000000000,1040.000000000000,1030.000000000000,16407.593124061514,1020.000000000000,1040.000000000000,1036.433235515845
2,ETH,2,1,10.000000000000,0.000355871886,0.000497512438,0.000476190476,8.166987349858,0.000355871886,0.000497512438,0.000452826759
3,USDC,3,1,10000.000000000000,1210.000000000000,1510.000000000000,1250.000000000000,13947.381198016597,1210.000000000000,1510.000000000000,1265.968579328206
4,ETH,4,2,10.000000000000,0.000450450450,0.000495049505,0.000454545455,5.808453496662,0.000450450450,0.000495049505,0.000452826759
5,USDC,5,2,10000.000000000000,1050.000000000000,1060.000000000000,1055.000000000000,19238.885565248278,1050.000000000000,1060.000000000000,1059.629997777449
6,ETH,6,3,10.000000000000,0.000485436893,0.000492610837,0.000491400491,0E-12,0.000485436893,0.000492610837,0.000485436893
7,USDC,7,3,10000.000000000000,1055.000000000000,1750.000000000000,1700.000000000000,30474.618433563053,1055.000000000000,1750.000000000000,1750.000000000000
8,ETH,8,4,10.000000000000,0.000480769231,0.000490196078,0.000487804878,0E-12,0.000480769231,0.000490196078,0.000480769231
9,USDC,9,4,10000.000000000000,1040.000000000000,1250.000000000000,1050.000000000000,30649.455198624491,1040.000000000000,1250.000000000000,1070.800866245103


# Initialize the Simulator and create the orders
Here the orders are entered into the simulator and the prices for one side of the strategy are flipped to coincide with the price convention

In [15]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

FastSim, initial_orders_import = setup_initial_orders(strats, sourceToken, targetToken)
initial_orders_import

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,USDCETH,ETH,12.95803,10.00000,ETH,False,0.00050,0.00040,0.00048,ETH per USDC,1
1,1,USDCETH,USDC,19951.57300,10000.00000,USDC,False,0.00096,0.00098,0.00097,ETH per USDC,0
2,2,USDCETH,ETH,11.63396,10.00000,ETH,False,0.00050,0.00036,0.00048,ETH per USDC,3
3,3,USDCETH,USDC,71432.10220,10000.00000,USDC,False,0.00066,0.00083,0.00080,ETH per USDC,2
4,4,USDCETH,ETH,106.58192,10.00000,ETH,False,0.00050,0.00045,0.00045,ETH per USDC,5
5,5,USDCETH,USDC,19976.33126,10000.00000,USDC,False,0.00094,0.00095,0.00095,ETH per USDC,4
6,6,USDCETH,ETH,12.02214,10.00000,ETH,False,0.00049,0.00049,0.00049,ETH per USDC,7
7,7,USDCETH,USDC,10687.91421,10000.00000,USDC,False,0.00057,0.00095,0.00059,ETH per USDC,6
8,8,USDCETH,ETH,13.38225,10.00000,ETH,False,0.00049,0.00048,0.00049,ETH per USDC,9
9,9,USDCETH,USDC,200831.31063,10000.00000,USDC,False,0.00080,0.00096,0.00095,ETH per USDC,8


# Parse the actions to perform the trades

In [16]:
parse_actions(FastSim, actionList, strats, is_by_target, sourceToken, targetToken)

FastSim.state()['trades']

Trading by Target: True; Source Token is USDC; Target Token is ETH


,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #6,False,True,None,10.00000,ETH,20474.61845,USDC,USDCETH,6,1,0.00049,ETH per USDC
0,0,0,A,AMM sells 10ETH buys 20475USDC,True,True,None,10.00000,ETH,20474.61845,USDC,USDCETH,[6],1,0.00049,ETH per USDC
0,1.0,1,0,route #8,False,True,None,10.00000,ETH,20649.45519,USDC,USDCETH,8,1,0.00048,ETH per USDC
0,1,1,A,AMM sells 10ETH buys 20649USDC,True,True,None,10.00000,ETH,20649.45519,USDC,USDCETH,[8],1,0.00048,ETH per USDC
0,2.0,2,0,route #0,False,True,None,2.97544,ETH,6407.59313,USDC,USDCETH,0,1,0.00046,ETH per USDC
0,2,2,A,AMM sells 3ETH buys 6408USDC,True,True,None,2.97544,ETH,6407.59313,USDC,USDCETH,[0],1,0.00046,ETH per USDC
0,3.0,3,0,route #2,False,True,None,1.83301,ETH,3947.38120,USDC,USDCETH,2,1,0.00046,ETH per USDC
0,3,3,A,AMM sells 2ETH buys 3947USDC,True,True,None,1.83301,ETH,3947.38120,USDC,USDCETH,[2],1,0.00046,ETH per USDC
0,4.0,4,0,route #4,False,True,None,4.19155,ETH,9238.88556,USDC,USDCETH,4,1,0.00045,ETH per USDC
0,4,4,A,AMM sells 4ETH buys 9239USDC,True,True,None,4.19155,ETH,9238.88556,USDC,USDCETH,[4],1,0.00045,ETH per USDC


# Display the status of the orders post-trade

In [17]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

final_orders_import = FastSim.state()['orders']
final_orders_import

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,USDCETH,ETH,12.95803,7.02456,ETH,False,0.00050,0.00040,0.00045,ETH per USDC,1
1,1,USDCETH,USDC,19951.57300,16407.59313,USDC,False,0.00096,0.00098,0.00096,ETH per USDC,0
2,2,USDCETH,ETH,11.63396,8.16699,ETH,False,0.00050,0.00036,0.00045,ETH per USDC,3
3,3,USDCETH,USDC,71432.10220,13947.38120,USDC,False,0.00066,0.00083,0.00079,ETH per USDC,2
4,4,USDCETH,ETH,106.58192,5.80845,ETH,False,0.00050,0.00045,0.00045,ETH per USDC,5
5,5,USDCETH,USDC,19976.33126,19238.88556,USDC,False,0.00094,0.00095,0.00094,ETH per USDC,4
6,6,USDCETH,ETH,12.02214,0.00000,ETH,False,0.00049,0.00049,0.00049,ETH per USDC,7
7,7,USDCETH,USDC,30474.61845,30474.61845,USDC,False,0.00057,0.00095,0.00057,ETH per USDC,6
8,8,USDCETH,ETH,13.38225,0.00000,ETH,False,0.00049,0.00048,0.00048,ETH per USDC,9
9,9,USDCETH,USDC,200831.31063,30649.45519,USDC,False,0.00080,0.00096,0.00093,ETH per USDC,8
